In [ ]:
!pip install opencv-python
!pip install ultralytics

In [ ]:
import os
import cv2
import json
from pathlib import Path
from ultralytics import YOLO

In [ ]:
# Pfad zu den Bildern und dem Ausgabeordner
input_folder = "   "  # Pfad zum Eingabeordner mit Bildern
output_folder = "   "  # Pfad zum Ausgabeordner
os.makedirs(output_folder, exist_ok=True)  # Ordner erstellen, falls nicht vorhanden

In [ ]:
# YOLO-Modell laden
model = YOLO("   ")  # Pfad zum YOLO-Modell .pt

In [ ]:
# Dictionary zum Speichern der Ergebnisse
results_dict = {}

# Alle Bilder im Eingabeordner durchgehen
for image_file in Path(input_folder).rglob("*.webp"): # Bildformat beachten
    # Bild laden
    img = cv2.imread(str(image_file))
    
    # YOLO Detections
    results = model(img)

    # Anzahl der Detections für das aktuelle Bild
    num_detections = sum(len(result.boxes) for result in results)

    # Ergebnisse im Dictionary speichern
    results_dict[str(image_file.name)] = num_detections
    
    # Über alle Ergebnisse iterieren
    for result in results:
        # Box-Koordinaten extrahieren
        detections = result.boxes.xyxy.cpu().numpy()
        
        # Originaler Dateiname und Komponenten extrahieren
        original_name = image_file.stem  # Dateiname ohne Erweiterung
        components = original_name.split("_")
        
        # Buchname und Seitennummer ermitteln
        book_number = "_".join(components[:-2])  # Buchnummer
        page_number = components[-2]  # Seitennummer
        illustration_number = components[-1]  # Illustrationsnummer
        
        # Detections speichern
        for i, (x1, y1, x2, y2) in enumerate(detections):
            # Zuschnitt der Detection
            cropped_img = img[int(y1):int(y2), int(x1):int(x2)]
            
            # Neuer Dateiname
            detection_number = f"{i:02d}"
            new_filename = f"{book_number}_{page_number}_{illustration_number}_{detection_number}.webp"
            
            # Speicherpfad
            output_path = os.path.join(output_folder, new_filename)
            
            # Speicher das ausgeschnittene Bild
            cv2.imwrite(output_path, cropped_img)

# Ergebnisse in einer JSON-Datei speichern
with open("   ", "w") as json_file:  # Pfad zur Ausgabe-JSON-Datei
    json.dump(results_dict, json_file, indent=4)

print("Alle Bilder wurden verarbeitet und die Ergebnisse wurden gespeichert.")